# [TR-004] Analyticity (WIP)

<!-- cspell:ignore cstride darkred labelpad matexpr rstride zorder -->

```{warning}
This report is Work-in-Progress.
```

In [ ]:
%matplotlib widget
import os
import warnings

import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from ampform.dynamics import (
    phase_space_factor,
    relativistic_breit_wigner_with_ff,
)
from IPython.display import Math
from ipywidgets import widgets
from matplotlib import cm
from mpl_interactions import heatmap_slicer

warnings.filterwarnings("ignore")

plt.rcParams.update({"font.size": 14})

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

## 

## Branch points of $\rho(s)$

Investigation of Section 2.1.2 in {cite}`aitchisonUnitarityAnalyticityCrossing2015`.

In [ ]:
s = sp.Symbol("s")
m1, m2 = sp.symbols("m1 m2", real=True)
rho = 16 * sp.pi * phase_space_factor(s, m1, m2)
rho

Or, assuming both decay products to be of unit mass:

In [ ]:
rho.subs(
    {
        m1: 1,
        m2: 1,
    }
)

In [ ]:
np_rho = sp.lambdify((s, m1, m2), rho, "numpy")

m1_val = 1.8
m2_val = 0.5
s_thr = (m1_val + m2_val) ** 2
s_diff = abs(m1_val - m2_val) ** 2

x = np.linspace(-1, +7, num=100)
y = np.linspace(-2, +2, num=100)
X, Y = np.meshgrid(x, y)
s_values = X + Y * 1j
rho_values = np_rho(s_values, m1=m1_val, m2=m2_val)

In [ ]:
rho_min, rho_max = -5, +5
Z = rho_values.real
fig, ax = plt.subplots(
    figsize=(10, 10),
    subplot_kw={"projection": "3d"},
    tight_layout=True,
)
fig.suptitle(R"Solution sheet for $\rho$ in the $s$-plane")
colors = np.select(
    [Z < rho_min, Z < 0, Z >= 0, Z > rho_max],
    [rho_min, Z / np.abs(rho_min), Z / np.abs(rho_max), rho_max],
)
colors = (colors + 1) / 2
ax.plot_surface(
    X,
    Y,
    Z,
    facecolors=cm.bwr(colors),
    linewidth=0,
)
ax.set_xlabel("Re($s$)")
ax.set_ylabel("Im($s$)")
ax.set_zlabel(R"Re($\rho$)")
ax.set_zlim(rho_min, rho_max)
ax.view_init(elev=25, azim=-110)
plt.show()

In [ ]:
if STATIC_WEB_PAGE:
    from IPython.display import Image

    output_file = "004-3D-plot.png"
    plt.savefig(output_file)
    display(Image(output_file))

In [ ]:
fig, axes = heatmap_slicer(
    x,
    y,
    (rho_values.real, rho_values.imag),
    heatmap_names=(R"Re($\rho$)", R"Im($\rho$)"),
    labels=("Re($s$)", "Im($s$)"),
    interaction_type="move",
    slices="both",
    vmin=-5,
    vmax=5,
    figsize=(12, 3),
)
for ax in axes[2:]:
    ax.set_ylim(rho_min, rho_max)
    tick_width = 5
    tick_min = np.around(rho_min / tick_width, decimals=0) * tick_width
    ax.set_yticks(np.arange(tick_min, rho_max + 0.1, 5))
axes[2].set_title("Re($s$)")
axes[3].set_title("Im($s$)")
for ax in axes[:3]:
    ax.axvline(s_diff, c="black", linewidth=0.3, linestyle="dotted")
    ax.axvline(s_thr, c="black", linewidth=0.3, linestyle="dotted")
for ax in axes:
    ax.axvline(0, c="black", linewidth=0.5)
    ax.axhline(0, c="black", linewidth=0.5)
axes[3].axvline(0, c="black", linewidth=0.5)
plt.show()

{{ run_interactive }}

In [ ]:
if STATIC_WEB_PAGE:
    from IPython.display import Image

    output_file = "004-interactive-locator.png"
    plt.savefig(output_file)
    display(Image(output_file))

## Physical vs. unphysical sheet

Interactive reproduction of Figure 49.1 on {pdg-review}`2020; Resonances; p.2`. See {func}`~ampform.dynamics.relativistic_breit_wigner_with_ff` for the origin of the parametrizations below.

In [ ]:
s = sp.Symbol("s")
m0, w0, m1, m2 = sp.symbols("m0 Gamma0 m1 m2", real=True, positive=True)

amp_unphysical = relativistic_breit_wigner_with_ff(
    s, m0, w0, m1, m2, angular_momentum=0, meson_radius=1
).doit()
Math(fR"\mathrm{{Unphysical:}} \quad {sp.latex(amp_unphysical)}")

In [ ]:
sub_expr = amp_unphysical.args[2].args[0].args[2]
amp_physical = amp_unphysical.subs(sub_expr, sp.sqrt(sub_expr ** 2))
Math(fR"\mathrm{{Physical:}} \quad {sp.latex(amp_physical)}")

In [ ]:
np_amp_physical = sp.lambdify((s, m0, w0, m1, m2), amp_physical, "numpy")
np_amp_unphysical = sp.lambdify((s, m0, w0, m1, m2), amp_unphysical, "numpy")

x_min, x_max = -0.2, 1.3
y_min, y_max = -1.8, +1.8
z_min, z_max = -2.5, +2.5

x = np.linspace(x_min, x_max, num=50)
y_neg = np.linspace(y_min, -1e-4, num=30)
y_pos = np.linspace(1e-4, y_max, num=30)

X, Y_neg = np.meshgrid(x, y_neg)
X, Y_pos = np.meshgrid(x, y_pos)
s_values_neg = X + Y_neg * 1j
s_values_pos = X + Y_pos * 1j
y0 = np.zeros(len(x))

z_cut_min = 0.75 * z_min
z_cut_max = 0.75 * z_max
cut_off_min = np.vectorize(lambda z: z if z > z_cut_min else z_cut_min)
cut_off_max = np.vectorize(lambda z: z if z < z_cut_max else z_cut_max)

plot_style = dict(
    linewidth=0,
    alpha=0.7,
    antialiased=True,
    rstride=1,
    cstride=1,
)
axis_style = dict(
    c="black",
    linewidth=0.7,
    linestyle="dashed",
)

fig, axes = plt.subplots(
    ncols=2,
    figsize=(10, 6),
    subplot_kw={"projection": "3d"},
    tight_layout=True,
)
ax1, ax2 = axes
fig.suptitle(
    "$S$-wave Breit-Wigner ($L=0$) plotted over the complex $s$-plane"
)

m0_min = np.sign(x_min) * np.sqrt(np.abs(x_min))
m0_max = np.sign(x_max) * np.sqrt(np.abs(x_max))


@widgets.interact(
    m0=(m0_min, m0_max, 0.01),
    w0=(0.0, y_max, 0.01),
    m1=(1e-4, m0_max / 2, 0.01),
    m2=(1e-4, m0_max / 2, 0.01),
)
def plot(m0=0.8, w0=0.3, m1=1e-4, m2=1e-4):
    def plot_expression(ax, amp, neg_color="green"):
        ax.clear()
        z_values_neg = amp(s_values_neg, m0, w0, m1, m2)
        z_values_pos = amp(s_values_pos, m0, w0, m1, m2)
        Z_neg = cut_off_min(cut_off_max(z_values_neg.imag))
        Z_pos = cut_off_min(cut_off_max(z_values_pos.imag))
        z0 = amp(x + 1e-5j, m0, w0, m1, m2).imag

        ax.plot_surface(X, Y_neg, Z_neg, **plot_style, color=neg_color)
        ax.plot_surface(X, Y_pos, Z_pos, **plot_style, color="green")
        ax.plot(x, y0, z0, linewidth=3, c="darkred", zorder=8)

        ax.set_xlabel("Re($s$)", labelpad=-15)
        ax.set_ylabel("Im($s$)", labelpad=-15)
        ax.set_zlabel("Im($A$)", labelpad=-15)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_zticks([])
        ax.set_zlim(z_min, z_max)

    plot_expression(ax1, np_amp_physical)
    plot_expression(ax2, np_amp_unphysical, neg_color="gold")

    ax1.text(x_min, y_max, z_max / 2, "physical sheet", c="green")
    ax2.text(x_min, y_min, -z_max, "unphysical sheet", c="gold")

    fig.canvas.draw_idle()

{{ run_interactive }}

In [ ]:
if STATIC_WEB_PAGE:
    from IPython.display import SVG

    output_file = "004-physical-vs-unphysical.svg"
    plt.savefig(output_file)
    display(SVG(output_file))